A entrada desse procedimento é:
10 - Resoluções digitalizadas v1_0.xlsx

Os procedimentos realizados são:
- Extração do cabeçalho do texto digitalizado

O produto do código é:<br>
**15 - Resoluções (cabeçalho) v1_0.xlsx**
- PAG
- PROTOCOLO
- TEXTO_PDF
- RELATOR
- CABEÇALHO

In [1]:
import pandas as pd
import re

In [2]:
path_produtos = 'Dados\Produtos\\'
path_originais = 'Dados\Coletados\\'
path_ajustados = 'Dados\Ajustados\\'

In [5]:
df_resol = pd.read_excel(path_produtos+'10 - Resoluções digitalizadas v1_0.xlsx', index_col=0)
#df_proinfo = pd.read_excel(path_produtos+'ProInfo - Lista Consolidada v01_0.xlsx', usecols=['Protocolo','Relator'])
#df_proinfo.rename(columns={'Protocolo': 'PROTOCOLO'}, inplace=True)
#df_proinfo.set_index('PROTOCOLO', inplace=True)

In [6]:
df_resol.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2524 entries, 0 to 2523
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   PAG        2524 non-null   int64 
 1   PROTOCOLO  2524 non-null   object
 2   TEXTO_PDF  2524 non-null   object
 3   RELATOR    2524 non-null   object
dtypes: int64(1), object(3)
memory usage: 98.6+ KB


In [7]:
print(df_resol.head(30).to_markdown())

|    |   PAG | PROTOCOLO       | TEXTO_PDF                                                                                                            | RELATOR                            |
|---:|------:|:----------------|:---------------------------------------------------------------------------------------------------------------------|:-----------------------------------|
|  0 |  4344 | TCE/000006/2009 | 1                                                                                                                    | Carolina Matos                     |
|    |       |                 | TCE                                                                                                                  |                                    |
|    |       |                 | Gabinete da Conselheira Carolina Costa                                                                               |                                    |
|    |       |                 | PROCESSO: TCE/000006/2

## 1 Extração do cabeçalho da resolução

In [8]:
df_resol['CABEÇALHO'] = None

In [9]:
df_resol.describe(include=object)

,PROTOCOLO,TEXTO_PDF,RELATOR,CABEÇALHO
count,2524,2524,2524,0
unique,2524,2523,13,0
top,TCE/000006/2009,RESOLUÇÃO Nº 238/2014\nRELATOR: CONSELHEIRO GI...,Pedro Henrique Lino de Souza,NaN
freq,1,2,744,NaN


In [10]:
# Verificando porque TEXTO_PDF tem um texto repetido
# A mesma resolução vale para os dois convênios
print(df_resol[df_resol.duplicated(subset=['TEXTO_PDF'], keep=False)].to_markdown())

|     |   PAG | PROTOCOLO       | TEXTO_PDF                                                                                            | RELATOR               | CABEÇALHO   |
|----:|------:|:----------------|:-----------------------------------------------------------------------------------------------------|:----------------------|:------------|
|  49 |   392 | TCE/000110/2007 | RESOLUÇÃO Nº 238/2014                                                                                | Gildasio Penedo Filho |             |
|     |       |                 | RELATOR: CONSELHEIRO GILDÁSIO PENEDO FILHO                                                           |                       |             |
|     |       |                 | PROTOCOLO: TCE/000110/2007 e TCE/001234/2007                                                         |                       |             |
|     |       |                 | NATUREZA: RECURSOS ATRIBUÍDOS A ENTIDADES E INSTITUIÇÕES                                   

In [14]:
# Processo para separar o cabeçalho
# Verificando uma situação de cada vez
# Palavras limite: Ementa, Vistos e Voto

def ExtraiCabeçalho(df_resol, termo):
    padrão = r'(.*)'+ termo
    for ind in df_resol.index:
        if pd.isna(df_resol.loc[ind,'CABEÇALHO']):
            exp = re.search(padrão, df_resol.loc[ind,'TEXTO_PDF'], re.DOTALL|re.IGNORECASE|re.ASCII )
            if exp:
                df_resol.loc[ind,'CABEÇALHO'] = exp.group(1)
    return df_resol

ExtraiCabeçalho(df_resol, "\nEmenta")
ExtraiCabeçalho(df_resol, "\nVistos")
ExtraiCabeçalho(df_resol, "\nVoto")
ExtraiCabeçalho(df_resol, "\nTRIBUNAL")
print(df_resol.loc[df_resol['CABEÇALHO'].isna()].to_markdown())
print(df_resol['CABEÇALHO'].isna().value_counts())



| PAG   | PROTOCOLO   | TEXTO_PDF   | RELATOR   | CABEÇALHO   |
|-------|-------------|-------------|-----------|-------------|
CABEÇALHO
False    2524
Name: count, dtype: int64


In [15]:
df_resol.to_excel(path_ajustados+'15 - Resoluções (cabeçalho) v1_0.xlsx')